In [1]:
import numpy as np
import pickle
from scipy.linalg import sqrtm, inv
import importlib
import time
import sys

def load_solver(pth, name):
    # load the serialized problem formulation
    with open(pth + '/' + name + '/problem.pickle', 'rb') as f:
        problem = pickle.load(f)
        print(f'Problem {name} loaded from path: {'../' + pth}')
    return problem

def load_parameters(pth, name):
    pars = np.load(pth + name)
    print(f'Parameters {name} loaded from path: {'../' + pth}')
    return pars

def set_parameters(model, data, problem):
    # assign parameter values
    problem.param_dict['Q_v_inv_sqrt'].value = sqrtm( inv(model['Q_v']) )
    problem.param_dict['Q_w_inv_sqrt'].value = sqrtm( inv(model['Q_w']) )
    problem.param_dict['Q_u_inv_sqrt'].value = sqrtm( inv(model['Q_u'])  )
    problem.param_dict['A'].value = model['A']
    problem.param_dict['B'].value = model['B']
    problem.param_dict['C'].value = model['C']
    problem.param_dict['x0'].value = np.zeros(model['n_states'])
    problem.param_dict['y'].value = data['y']

def reload_module(module_name):
    if module_name in sys.modules:
        del sys.modules[module_name]
    importlib.import_module(module_name)

def import_cpg_solver(problem, pth):
    module_path = f'{pth}.{problem}.cpg_solver'
    # Dynamically import the module
    cpg_solver_module = importlib.import_module(module_path)

    # Get the cpg_solve function from the module
    cpg_solve = getattr(cpg_solver_module, 'cpg_solve')
    return cpg_solve

def solve_cvxpy(problem):
    # solve problem with CVXPY
    t0 = time.time()
    val = problem.solve(eps_abs=1e-3, eps_rel=1e-3, max_iter=4000, polish=False)
    t1 = time.time()
    print('\nCVXPY\nSolve time: %.3f ms' % (1000 * (t1 - t0)))
    print('Objective function value: %.6f\n' % val)

def solve_cvxpygen(problem, cpg_solve):
    # solve problem with C code via python wrapper
    problem.register_solve('CPG', cpg_solve)
    t0 = time.time()
    val = problem.solve(method='CPG')
    t1 = time.time()
    print('\nCVXPYgen\nSolve time: %.3f ms' % (1000 * (t1 - t0)))
    print('Objective function value: %.6f\n' % val)
    


In [2]:
# load data
pth = '../datasets/'
data_lumped_element_2 = load_parameters(pth, 'data_lumped_element_2.npz')
data_lumped_element_4 = load_parameters(pth, 'data_lumped_element_4.npz')
data_lumped_element_8 = load_parameters(pth, 'data_lumped_element_8.npz')

# laod models
pth = '../models/'
model_lumped_element_2 = load_parameters(pth, 'lumped_element_2.npz')
model_lumped_element_4 = load_parameters(pth, 'lumped_element_4.npz')
model_lumped_element_8 = load_parameters(pth, 'lumped_element_8.npz')

# load problems
pth = '../codegen/'
mh_sise_tikhonov_2 = load_solver(pth, 'mh_sise_tikhonov_2')
mh_sise_tikhonov_4 = load_solver(pth, 'mh_sise_tikhonov_4')
mh_sise_tikhonov_8 = load_solver(pth, 'mh_sise_tikhonov_8')

Parameters data_lumped_element_2.npz loaded from path: ../../datasets/
Parameters data_lumped_element_4.npz loaded from path: ../../datasets/
Parameters data_lumped_element_8.npz loaded from path: ../../datasets/
Parameters lumped_element_2.npz loaded from path: ../../models/
Parameters lumped_element_4.npz loaded from path: ../../models/
Parameters lumped_element_8.npz loaded from path: ../../models/
Problem mh_sise_tikhonov_2 loaded from path: ../../codegen/
Problem mh_sise_tikhonov_4 loaded from path: ../../codegen/
Problem mh_sise_tikhonov_8 loaded from path: ../../codegen/


In [3]:

# set solver parameters
set_parameters(model=model_lumped_element_2, data=data_lumped_element_2, problem=mh_sise_tikhonov_2)
set_parameters(model=model_lumped_element_4, data=data_lumped_element_4, problem=mh_sise_tikhonov_4)
set_parameters(model=model_lumped_element_8, data=data_lumped_element_8, problem=mh_sise_tikhonov_8)


In [4]:
# ########## CVXPY ###########
solve_cvxpy(mh_sise_tikhonov_2)
solve_cvxpy(mh_sise_tikhonov_4)
solve_cvxpy(mh_sise_tikhonov_8)

# ########## CVXPYGEN ###########
cpg_solve_2 = import_cpg_solver(problem='mh_sise_tikhonov_2', pth='codegen')
cpg_solve_4 = import_cpg_solver(problem='mh_sise_tikhonov_4', pth='codegen')
cpg_solve_8 = import_cpg_solver(problem='mh_sise_tikhonov_8', pth='codegen')

solve_cvxpygen(mh_sise_tikhonov_2, cpg_solve_2)
solve_cvxpygen(mh_sise_tikhonov_4, cpg_solve_4)
solve_cvxpygen(mh_sise_tikhonov_8, cpg_solve_8)


CVXPY
Solve time: 28.006 ms
Objective function value: 14.754511


CVXPY
Solve time: 43.009 ms
Objective function value: 15.436318


CVXPY
Solve time: 142.033 ms
Objective function value: 30.532575



ImportError: generic_type: type "cpg_params" is already registered!